# Problem 1
#### visit the website https://www.flipkart.com/ then choose anyproductfind out the review from the productand arrange in the csv file(name of person, rating, title, review) finally create a end to end application from the above usecase and show case the data over the html page the before next session you can complete it by friday you can submit to my mail id and keep in cc krish sir sunny.savita@ineuron.ai krish.naik@ineuron.ai

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
import logging
import cx_Oracle
from flask import Flask, render_template, request,jsonify
from flask_cors import CORS,cross_origin
import logging
logging.basicConfig(filename="scrapper.log" , level=logging.INFO)
import os

application = Flask(__name__)
app=application

@app.route("/", methods = ['GET'])
def homepage():
    return render_template("index.html")

@app.route("/review" , methods = ['POST' , 'GET'])
def index():
    try:
        if request.method == 'POST':
            try:
                query = request.form['content'].replace(" ","")
                connection_string = 'deep_db/1122@localhost:1521/xe'
                connection = cx_Oracle.connect(connection_string)

                query = 'apple-iphone-14-midnight-256-gb'
                headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}
                response = requests.get(f"https://www.flipkart.com/{query}/product-reviews/itmdb32e3c997112?pid=MOBGHWFH4H3MMRAA&lid=LSTMOBGHWFH4H3MMRAAO7KNHD&marketplace=FLIPKART")
                soup = BeautifulSoup(response.content, "html.parser")
                review_elements = soup.find_all("div", class_="col _2wzgFH K0kLPL")
                reviews = []
                for i in review_elements:
                    reviewer_name = i.find("p", class_="_2sc7ZR _2V5EHH").text.strip()
                    location = i.find("p", class_="_2mcZGG").text[17:]
                    title = i.find("p", class_="_2-N8zT").text.strip()
                    desc = i.find("div", class_="t-ZTKy").text[:-9]
                    rate = int(i.find("div", class_="_3LWZlK _1BLPMq").text)

                    # ==========================================================
                    sql_query = "SELECT object_name from all_objects where upper(object_name) = 'INS_DB_TEST_PY' and upper(object_type)='TABLE'"
                    with connection.cursor() as cursor:
                        cursor.execute(sql_query)
                        result = cursor.fetchall()
                        if len(result) == 0:
                            sql_query1 = 'create table INS_DB_TEST_PY(name varchar2(35), location varchar2(100), title varchar2(1000), rate number, description varchar2(1000))'
                            cursor.execute(sql_query1)
                            print('Table created')
                        else:
                            sql_query1 = "INSERT INTO INS_DB_TEST_PY (name, location, title, rate, discription) VALUES (:1, :2, :3, :4, :5)"
                            data_to_insert = (reviewer_name, location, title, rate, desc)
                            cursor.execute(sql_query1, data_to_insert)
                            connection.commit()
                    # ============================================================
                    review_data = {
                        "Reviewer Name": reviewer_name,
                        # "Review Date": review_date,
                        "Location": location,
                        "Title": title,
                        "Rate": rate,
                        "Description": desc
                    }
                    reviews.append(review_data)
                    #print('the type is ',  type(review_data))
                """for review in reviews:
                    print(review)"""
            finally:
                if 'connection' in locals() and connection:
                    connection.close()
                    print("commited.")
            print('deep')
            """headers = ['Name', 'Location', 'Title','Rate','description']
            with open('Data.csv', 'w') as file:
                writer = csv.DictWriter(file, fieldnames=headers)
                writer.writerows(reviews)"""
            return render_template('result.html', reviews=reviews)
        else:
            print('narayan')
            return render_template('index.html')
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        return "Internal Server Error", 500

if __name__ == "__main__":
    app.run()
    

 * Serving Flask app '__main__'
 * Debug mode: off
commited.
deep
